In [ ]:
import os
import cv2
import glob
import ntpath
import pandas as pd

In [ ]:
generate_nucleus_from_images()

In [ ]:
counter = 0

def generate_nucleus_from_images():

    for grade_no in range(0, 5):
        
        src_path = "G:/LiverCancer/original/" + "grade" + str(grade_no) + "/"
        src_files = glob.glob(src_path + "*.jpg")
    
        for src_file in src_files:

            print(src_file)
            
            generate_nucleus_from_image(src_file)

def generate_nucleus_from_image(src_file):
    
    dst_path = "G:/LiverCancer/nucleus/image/"

    global counter
    
    if "grade0" in src_file: sample_label = 0
    if "grade1" in src_file: sample_label = 1
    if "grade2" in src_file: sample_label = 2
    if "grade3" in src_file: sample_label = 3
    if "grade4" in src_file: sample_label = 4

    img = cv2.imread(src_file)
    df_labels = pd.read_csv("G:/LiverCancer/nucleus/_labels/" + str(counter) + ".csv")
    
    for index, annotation in df_labels.iterrows():
        
        index = annotation["index"]
        coord_x = annotation["coord_x"]
        coord_y = annotation["coord_y"]
        diameter = annotation["diameter"]
    
        box_img = get_square_from_img(img, coord_x, coord_y, 128)
      
        os.makedirs(dst_path + "grade" + str(sample_label) + "/" + ntpath.basename(src_file) + "/", exist_ok=True)

        file_name = "grade_" + str(sample_label) + "_" + ntpath.basename(src_file)[-6:-4]  + "_" + str(index) + "_" + str(coord_x) + "_" + str(coord_y) + "_" + str(diameter) + ".jpg"

        cv2.imwrite(dst_path + "grade" + str(sample_label) + "/" + ntpath.basename(src_file) + "/" + file_name, box_img)

    counter += 1

def get_square_from_img(img, center_x, center_y, box_size):

    start_x = max(center_x - box_size / 2, 0) if center_x + box_size / 2 <= img.shape[0] else img.shape[0] - box_size
    start_y = max(center_y - box_size / 2, 0) if center_y + box_size / 2 <= img.shape[1] else img.shape[1] - box_size

    start_x = int(start_x)
    start_y = int(start_y)

    res = img[start_x:start_x + box_size, start_y:start_y + box_size]

    return res